# Project: Build a QA RAG System on Asthma - Source is clevelandclinic

## Install OpenAI, and LangChain dependencies

In [105]:
!pip install langchain==0.3.11
!pip install langchain-openai==0.2.12
!pip install langchain-community==0.3.11
!pip install langchain-chroma==0.1.4
!pip install sentence-transformers==2.7.0

%%capture
!pip install unstructured selenium textstat tiktoken
!pip install pyngrok==7.1.5
!pip install streamlit==1.32.2


  Using cached tokenizers-0.20.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached tokenizers-0.20.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.47.1 requires tokenizers<0.22,>=0.21, but you have tokenizers 0.20.3 which is incompatible.
  Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Succe

UsageError: Line magic function `%%capture` not found.


In [106]:
%%capture
!pip install openai unstructured selenium textstat tiktoken


In [107]:
!pip install streamlit

In [108]:
!pip install pyngrok==7.1.5

In [109]:
##%%writefile app.py

Overwriting app.py


In [110]:
import streamlit as st

In [111]:
import tiktoken
import matplotlib.pyplot as plt
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
#from textstat import flesch_reading_ease
from collections import Counter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader, SeleniumURLLoader
from langchain_chroma import Chroma
#from langchain_community.vectorstores.chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_message_histories import StreamlitChatMessageHistory
from langchain_core.callbacks.base import BaseCallbackHandler
from operator import itemgetter
import streamlit as st
import tempfile
import os
import pandas as pd
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

## Load Open AI API Credentials

In [112]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [113]:
import yaml

with open('chatgpt_api_credentials.yml', 'r') as file:
    api_creds = yaml.safe_load(file)

api_creds.keys()

dict_keys(['openai_key'])

In [114]:
import os

os.environ['OPENAI_API_KEY'] = api_creds['openai_key']
OPENAI_KEY = api_creds['openai_key']

In [ ]:
#from getpass import getpass
#OPENAI_KEY = getpass('Enter Open AI API Key: ')

Enter Open AI API Key: ··········


### Open AI Embedding Models

LangChain enables us to access Open AI embedding models which include the newest models: a smaller and highly efficient `text-embedding-3-small` model, and a larger and more powerful `text-embedding-3-large` model.

In [115]:
openai_embed_model = OpenAIEmbeddings(
    model='text-embedding-3-small',
    openai_api_key=OPENAI_KEY
)

In [120]:
# Customize initial app landing page

st.set_page_config(page_title="File QA Chatbot", page_icon="🤖")
st.title("Welcome to File QA RAG Chatbot 🤖")

print("hi)")

Overwriting app.py


In [118]:
website = "https://my.clevelandclinic.org/health/diseases/6424-asthma"

In [119]:
def load_document(loader_class, website_url):
    loader = loader_class([website_url])
    return loader.load()

In [79]:
selenium_loader_doc = load_document(SeleniumURLLoader, website)

### Code to load multiple URLs via SeleniumURLLoader is stated below.

### However in this project I have used only one genuine source in the interest of time(as shown above).



In [ ]:
# urls = [
#     "https://my.clevelandclinic.org/health/diseases/6424-asthma",
#     "https://www.mayoclinic.org/diseases-conditions/asthma/diagnosis-treatment/drc-20369660",
#     "https://www.who.int/news-room/fact-sheets/detail/asthma"
#     ]

# # collect data using selenium url loader
# loader = SeleniumURLLoader(urls=urls)
# documents = loader.load()

# documentList = []
# for doc in documents:
#     d = str(doc.page_content).replace("\\n", " ").replace("\\t", " ").replace("\n", " ").replace("\t", " ")
#     documentList.append(d)

In [80]:
selenium_loader_doc[0]

Document(metadata={'source': 'https://my.clevelandclinic.org/health/diseases/6424-asthma', 'title': 'Asthma: Types, Causes, Symptoms, Diagnosis & Treatment', 'description': 'Asthma is a chronic disease that affects the airways of your lungs. It can be managed but not cured. ', 'language': 'en'}, page_content='Locations:\n\nAbu Dhabi|Canada|Florida|London|Nevada|Ohio|\n\nHome/\n\nHealth Library/\n\nDiseases & Conditions/\n\nAsthma\n\nAdvertisement\n\nAdvertisement\n\nAsthma\n\nBronchial asthma (or asthma) is a lung disease. Your airways get narrow and swollen and are blocked by excess mucus. Medications can treat these symptoms.\n\nAdvertisement\n\nCleveland Clinic is a non-profit academic medical center. Advertising on our site helps support our mission. We do not endorse non-Cleveland Clinic products or services. Policy\n\nCare at Cleveland Clinic\n\nAsthma Treatment for Adults\n\nFind a Doctor and Specialists\n\nAsthma Treatment for Children\n\nFind a Pediatric Doctor and Specialists

### Split larger documents into smaller chunks

In [81]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1450, chunk_overlap=200)
chunked_docs = splitter.split_documents(selenium_loader_doc)

## Vector Databases

One of the most common ways to store and search over unstructured data is to embed it and store the resulting embedding vectors, and then at query time to embed the unstructured query and retrieve the embedding vectors that are 'most similar' to the embedded query. A vector database takes care of storing embedded data and performing vector search for you.

### Chroma Vector DB

[Chroma](https://docs.trychroma.com/getting-started) is a AI-native open-source vector database focused on developer productivity and happiness. Chroma is licensed under Apache 2.0.

### Create a Vector DB and persist on disk

Here we initialize a connection to a Chroma vector DB client, and also we want to save to disk, so we simply initialize the Chroma client and pass the directory where we want the data to be saved to.

In [82]:
# create vector DB of docs and embeddings - takes < 30s on Colab
chroma_db = Chroma.from_documents(documents=chunked_docs,
                                  collection_name='rag_asthma_db',
                                  embedding=openai_embed_model,
                                  # need to set the distance function to cosine else it uses euclidean by default
                                  # check https://docs.trychroma.com/guides#changing-the-distance-function
                                  collection_metadata={"hnsw:space": "cosine"},
                                  persist_directory="./asthma_db")

### Load Vector DB from disk

This is just to show once you have a vector database on disk you can just load and create a connection to it anytime

In [83]:
# load from disk
chroma_db = Chroma(persist_directory="./asthma_db",
                   collection_name='rag_asthma_db',
                   embedding_function=openai_embed_model)

In [ ]:
chroma_db

## Load Connection to LLM

Here we create a connection to ChatGPT to use later in our chains

In [84]:
chatgpt = ChatOpenAI(model_name='gpt-4o-mini', temperature=0,openai_api_key=OPENAI_KEY)

## Chained Retrieval Pipeline

This strategy uses a chain of multiple retrievers sequentially to get to the most relevant documents. The following is the flow

Similarity Retrieval → Compression Filter → Reranker Model Retrieval




In [85]:
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain.retrievers.document_compressors import LLMChainFilter
from langchain.retrievers import ContextualCompressionRetriever

# Retriever 1 - simple cosine distance based retriever
similarity_retriever = chroma_db.as_retriever(search_type="similarity",
                                              search_kwargs={"k": 5})

#  decides which of the initially retrieved documents to filter out and which ones to return
_filter = LLMChainFilter.from_llm(llm=chatgpt)
# Retriever 2 - retrieves the documents similar to query and then applies the filter
compressor_retriever = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=similarity_retriever
)

# download an open-source reranker model - BAAI/bge-reranker-v2-m3
reranker = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-large")
reranker_compressor = CrossEncoderReranker(model=reranker, top_n=3)
# Retriever 3 - Uses a Reranker model to rerank retrieval results from the previous retriever
final_retriever = ContextualCompressionRetriever(
    base_compressor=reranker_compressor, base_retriever=compressor_retriever
)

In [86]:
query = "What asthma treatment options are there?"
docs = final_retriever.invoke(query)
docs

[Document(metadata={'description': 'Asthma is a chronic disease that affects the airways of your lungs. It can be managed but not cured. ', 'language': 'en', 'source': 'https://my.clevelandclinic.org/health/diseases/6424-asthma', 'title': 'Asthma: Types, Causes, Symptoms, Diagnosis & Treatment'}, page_content='Diagnosis and Tests\n\nHow do healthcare providers diagnose asthma?\n\nYour healthcare provider will review your medical history, including information about your parents and siblings. Your provider will also ask you about your symptoms. Your provider will need to know any history of allergies, eczema (a bumpy rash caused by allergies) and other lung diseases.\n\nYour provider may order spirometry. This test measures airflow through your lungs and is used to diagnose and monitor your progress with treatment. Your healthcare provider may order a chest X-ray, blood test or skin test.\n\nManagement and Treatment\n\nWhat asthma treatment options are there?\n\nYou have options to help

In [39]:
query = "How do healthcare providers diagnose asthma?"
docs = final_retriever.invoke(query)
docs

[Document(metadata={'description': 'Asthma is a chronic disease that affects the airways of your lungs. It can be managed but not cured. ', 'language': 'en', 'source': 'https://my.clevelandclinic.org/health/diseases/6424-asthma', 'title': 'Asthma: Types, Causes, Symptoms, Diagnosis & Treatment'}, page_content='Diagnosis and Tests\n\nHow do healthcare providers diagnose asthma?\n\nYour healthcare provider will review your medical history, including information about your parents and siblings. Your provider will also ask you about your symptoms. Your provider will need to know any history of allergies, eczema (a bumpy rash caused by allergies) and other lung diseases.\n\nYour provider may order spirometry. This test measures airflow through your lungs and is used to diagnose and monitor your progress with treatment. Your healthcare provider may order a chest X-ray, blood test or skin test.\n\nManagement and Treatment\n\nWhat asthma treatment options are there?\n\nYou have options to help

## Build a QA RAG Chain

To build a RAG chain we need a prompt template which instructs the LLM to not answer questions beyond the scope of the retrieved context documents, there are various such prompts out there, we craft one ourselves below

In [87]:
from langchain_core.prompts import ChatPromptTemplate

qa_template = """You are an assistant for question-answering tasks.
            Use the following pieces of retrieved context to answer the question.
            If no context is present or if you don't know the answer, just say that you don't know.
            Do not make up the answer unless it is there in the provided context.
            Give a detailed answer with regard to the question.

            Question:
            {question}

            Context:
            {context}

            Answer:
         """

qa_prompt = ChatPromptTemplate.from_template(qa_template)

In [88]:
# This function formats retrieved documents before sending to LLM
def format_docs(docs):
  return "\n\n".join([d.page_content for d in docs])

In [89]:
# Manages live updates to a Streamlit app's display by appending new text tokens
# to an existing text stream and rendering the updated text in Markdown
class StreamHandler(BaseCallbackHandler):
  def __init__(self, container, initial_text=""):
    self.container = container
    self.text = initial_text

  def on_llm_new_token(self, token: str, **kwargs) -> None:
    self.text += token
    self.container.markdown(self.text)

In [90]:
# Create a QA RAG System Chain
qa_rag_chain = (
  {
    "context": itemgetter("question") # based on the user question get context docs
      |
    final_retriever
      |
    format_docs,
    "question": itemgetter("question") # user question
  }
    |
  qa_prompt # prompt with above user question and context
    |
  chatgpt # above prompt is sent to the LLM for response
)

In [92]:
# Store conversation history in Streamlit session state
streamlit_msg_history = StreamlitChatMessageHistory(key="langchain_messages")

# Shows the first message when app starts
if len(streamlit_msg_history.messages) == 0:
  streamlit_msg_history.add_ai_message("Please ask your question?")

# Render current messages from StreamlitChatMessageHistory
for msg in streamlit_msg_history.messages:
  st.chat_message(msg.type).write(msg.content)

# Callback handler which does some post-processing on the LLM response
# Used to post the top 3 document sources used by the LLM in RAG response
class PostMessageHandler(BaseCallbackHandler):
  def __init__(self, msg: st.write):
    BaseCallbackHandler.__init__(self)
    self.msg = msg
    self.sources = []

  def on_retriever_end(self, documents, *, run_id, parent_run_id, **kwargs):
    source_ids = []
    for d in documents: # retrieved documents from retriever based on user query
      metadata = {
        "source": d.metadata["source"],
        "page": d.metadata["page"],
        "content": d.page_content[:200]
      }
      idx = (metadata["source"], metadata["page"])
      if idx not in source_ids: # store unique source documents
        source_ids.append(idx)
        self.sources.append(metadata)

  def on_llm_end(self, response, *, run_id, parent_run_id, **kwargs):
    if len(self.sources):
      st.markdown("__Sources:__ "+"\n")
      st.dataframe(data=pd.DataFrame(self.sources[:3]),
                    width=1000) # Top 3 sources


# If user inputs a new prompt, display it and show the response
if user_prompt := st.chat_input():
  st.chat_message("human").write(user_prompt)
  # This is where response from the LLM is shown
  with st.chat_message("ai"):
    # Initializing an empty data stream
    stream_handler = StreamHandler(st.empty())
    # UI element to write RAG sources after LLM response
    sources_container = st.write("")
    pm_handler = PostMessageHandler(sources_container)
    config = {"callbacks": [stream_handler, pm_handler]}
    # Get LLM response
    response = qa_rag_chain.invoke({"question": user_prompt},
                                    config)

2025-01-18 14:02:26.005 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 14:02:26.007 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 14:02:26.009 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 14:02:26.010 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 14:02:26.011 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 14:02:26.011 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 14:02:26.012 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-18 14:02:26.013 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

### START Streamlit app

In [149]:

!pip install langchain==0.3.11
!pip install langchain-openai==0.2.12
!pip install langchain-community==0.3.11
!pip install langchain-chroma==0.1.4
!pip install sentence-transformers==2.7.0


  Using cached tokenizers-0.20.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached protobuf-5.29.3-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
Using cached tokenizers-0.20.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
Using cached protobuf-5.29.3-cp38-abi3-manylinux2014_x86_64.whl (319 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.32.2 requires protobuf<5,>=3.20, but you have protobuf 5.29.3 which is incompatible.
tensorflow 2.17.1 require

In [150]:
%%capture
!pip install openai unstructured selenium textstat tiktoken

In [151]:
%%capture
!pip install unstructured selenium textstat tiktoken
!pip install pyngrok==7.1.5
!pip install streamlit==1.32.2

In [ ]:
#!pip install streamlit

#!pip install pyngrok==7.1.5

In [93]:
#!streamlit run app.py --server.port=8989 &>./logs.txt &

### ngrok URL

https://dashboard.ngrok.com/get-started/your-authtoken

In [104]:
from pyngrok import ngrok
import yaml

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken
# Get your authtoken from `ngrok_credentials.yml` file
with open('ngrok_credentials.yml', 'r') as file:
    NGROK_AUTH_TOKEN = yaml.safe_load(file)
ngrok.set_auth_token(NGROK_AUTH_TOKEN['ngrok_key'])

# Open an HTTPs tunnel on port XXXX which you get from your `logs.txt` file
ngrok_tunnel = ngrok.connect(8989)
#ngrok_tunnel = ngrok.connect(8501)
print("Streamlit App:", ngrok_tunnel.public_url)

Streamlit App: https://c8be-34-44-41-65.ngrok-free.app


In [133]:
!pip install streamlit
!pip install pyngrok==7.1.5


In [134]:
import streamlit as st

In [136]:
!lsof -i :8989

COMMAND     PID USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
streamlit 26826 root    6u  IPv4 714970      0t0  TCP *:8989 (LISTEN)
streamlit 26826 root    7u  IPv6 714971      0t0  TCP *:8989 (LISTEN)
streamlit 26826 root    9u  IPv4 719848      0t0  TCP localhost:8989->localhost:42740 (ESTABLISHED)
ngrok     27368 root    9u  IPv4 718771      0t0  TCP localhost:42740->localhost:8989 (ESTABLISHED)


In [142]:
!kill $(lsof -t -i:8989)

In [143]:

"""### START Streamlit app"""

!streamlit run app.py --server.port=8989 &>./logs.txt &

In [144]:
# Terminate open tunnels if exist
ngrok.kill()

In [145]:
from pyngrok import ngrok
import yaml


In [146]:
# Setting the authtoken
# Get your authtoken from `ngrok_credentials.yml` file
with open('ngrok_credentials.yml', 'r') as file:
    NGROK_AUTH_TOKEN = yaml.safe_load(file)
ngrok.set_auth_token(NGROK_AUTH_TOKEN['ngrok_key'])

# Open an HTTPs tunnel on port XXXX which you get from your `logs.txt` file
ngrok_tunnel = ngrok.connect(8989)
#ngrok_tunnel = ngrok.connect(8501)
print("Streamlit App:", ngrok_tunnel.public_url)

Streamlit App: https://088e-34-44-41-65.ngrok-free.app


In [152]:
# Install required packages if not already installed
!pip install streamlit
!pip install pyngrok==7.1.5

import streamlit as st
from pyngrok import ngrok
import yaml
import os
import time

# Kill any existing processes on port 8989
!kill $(lsof -t -i:8989) 2>/dev/null || true

# Start Streamlit in the background
os.system('streamlit run app.py --server.port=8989 &>./logs.txt &')

# Give Streamlit a moment to start up
time.sleep(5)

# Kill any existing ngrok processes
ngrok.kill()

# Set up ngrok
with open('ngrok_credentials.yml', 'r') as file:
    NGROK_AUTH_TOKEN = yaml.safe_load(file)
ngrok.set_auth_token(NGROK_AUTH_TOKEN['ngrok_key'])

# Create the tunnel
ngrok_tunnel = ngrok.connect(8989)
print("Streamlit App URL:", ngrok_tunnel.public_url)

Streamlit App URL: https://9f23-34-44-41-65.ngrok-free.app


In [174]:
!pip install streamlit
!pip install pyngrok==7.1.5
!pip install psutil
!pip install openai

import streamlit as st
from pyngrok import ngrok
import yaml
import os
import time
import requests
import psutil
import signal
from datetime import datetime
from openai import OpenAI

def log_to_colab(message):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{timestamp}] {message}")

# First, let's verify the API credentials
try:
    # Check OpenAI credentials
    with open("chatgpt_api_credentials.yml", "r") as file:
        api_creds = yaml.safe_load(file)
        OPENAI_KEY = api_creds["openai_key"]
    log_to_colab("OpenAI credentials loaded")

    # Check ngrok credentials
    with open("ngrok_credentials.yml", "r") as file:
        ngrok_creds = yaml.safe_load(file)
        NGROK_TOKEN = ngrok_creds["ngrok_key"]
    log_to_colab("Ngrok credentials loaded")
except Exception as e:
    log_to_colab(f"Error loading credentials: {str(e)}")
    raise e

# Test OpenAI API
def test_openai():
    try:
        client = OpenAI(api_key=OPENAI_KEY)
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": "Hello!"}]
        )
        log_to_colab("OpenAI API test successful")
        return True
    except Exception as e:
        log_to_colab(f"OpenAI API test failed: {str(e)}")
        return False

# Port management functions
def kill_process_on_port(port):
    try:
        if os.name == 'posix':
            os.system(f'lsof -ti:{port} | xargs kill -9')
        else:
            os.system(f'for /f "tokens=5" %a in (\'netstat -aon ^| find ":{port}" ^| find "LISTENING"\') do taskkill /f /pid %a')
        time.sleep(2)
        log_to_colab(f"Killed process on port {port}")
    except Exception as e:
        log_to_colab(f"Error killing process on port {port}: {e}")

def is_port_available(port):
    import socket
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    try:
        sock.bind(('localhost', port))
        sock.close()
        return True
    except:
        return False

# Main execution
try:
    # 1. Test OpenAI API
    if not test_openai():
        raise Exception("OpenAI API test failed")

    # 2. Kill existing processes
    ngrok.kill()
    log_to_colab("Killed existing ngrok processes")

    # 3. Set up port
    port = 8501  # Using Streamlit's default port
    kill_process_on_port(port)

    if not is_port_available(port):
        log_to_colab(f"Port {port} is not available, trying alternative...")
        port = 8502
        if not is_port_available(port):
            raise Exception("No available ports found")

    log_to_colab(f"Using port {port}")

    # 4. Start Streamlit
    log_to_colab("Starting Streamlit...")
    os.system(f'streamlit run app.py --server.port={port} &>./logs.txt &')
    time.sleep(5)  # Give Streamlit time to start

    def monitor_logs():
      #"""Monitor both Streamlit and app logs"""
      try:
          print("\nStreamlit logs:")
          !tail -f logs.txt &
          print("\nChatbot debug logs:")
          !tail -f chatbot_debug.log &
      except Exception as e:
        print(f"Error monitoring logs: {e}")

    # 5. Set up ngrok
    ngrok.set_auth_token(NGROK_TOKEN)
    log_to_colab("Ngrok authentication successful")

    # 6. Create tunnel
    ngrok_tunnel = ngrok.connect(port)
    log_to_colab(f"Streamlit App URL: {ngrok_tunnel.public_url}")



    # 7. Monitor logs
    log_to_colab("Starting log monitoring...")
    !tail -f logs.txt &
    !tail -f chatbot_debug.log &

except Exception as e:
    log_to_colab(f"Setup failed: {str(e)}")
    raise e


monitor_logs()


# Add this to test the retriever directly
def test_retriever():
    try:
        test_query = "What is asthma?"
        docs = final_retriever.invoke(test_query)
        print(f"Retrieved {len(docs)} documents")
        print("First document preview:", docs[0].page_content[:100])
    except Exception as e:
        print(f"Retriever test failed: {e}")

# Test the retriever
test_retriever()

!tail -f chatbot_debug.log

# Keep the notebook running
while True:
    try:
        time.sleep(1)
        # Check if Streamlit is still running
        response = requests.get(f"http://localhost:{port}")
    except KeyboardInterrupt:
        log_to_colab("Shutting down...")
        ngrok.kill()
        break
    except:
        pass

[2025-01-18 19:31:04] OpenAI credentials loaded
[2025-01-18 19:31:04] Ngrok credentials loaded
[2025-01-18 19:31:04] OpenAI API test successful
[2025-01-18 19:31:04] Killed existing ngrok processes
[2025-01-18 19:31:06] Killed process on port 8501
[2025-01-18 19:31:06] Using port 8501
[2025-01-18 19:31:06] Starting Streamlit...
[2025-01-18 19:31:11] Ngrok authentication successful
[2025-01-18 19:31:11] Streamlit App URL: https://8188-34-44-41-65.ngrok-free.app
[2025-01-18 19:31:11] Starting log monitoring...


KeyboardInterrupt: 

In [ ]:
# Add this to your Colab notebook
# Display the contents of the log file
#!cat chatbot_debug.log

# Or to see the last few lines
!tail -n 50 chatbot_debug.log

# To continuously monitor the log file (useful while using the chatbot)
##!tail -f chatbot_debug.log

In [ ]:
!cat chatbot_debug.log

start here